load data and check the nature of the height and weight columns

In [47]:
import pandas as pd
import re
import matplotlib.pyplot as plt



# Load your dataset (adjust filename if needed)
df = pd.read_csv("fifa21_raw_data.csv")

# Print the first few rows of the DataFrame
# print(df[['Height', 'Weight']].head(10))


/tmp/ipykernel_1992/1864073130.py:8: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("fifa21_raw_data.csv")


1. covert the heights to numerical values

In [ ]:

def convert_height(h):
    if pd.isna(h):
        return None
    h = str(h).strip().lower()
    
    if "'" in h:  # e.g. 5'7
        parts = re.findall(r'\d+', h)
        if len(parts) == 2:
            feet, inches = map(int, parts)
            return round((feet * 30.48) + (inches * 2.54))
    elif "cm" in h:  # e.g. 180cm
        return int(re.sub(r'[^0-9]', '', h))
    
    return None

df['Height_cm'] = df['Height'].apply(convert_height)
print(df[['Height_cm']].head(10))


1. convert the weight to numerical values

In [10]:
def convert_weight(w):
    if pd.isna(w):
        return None
    w = str(w)
    if "lbs" in w:
        lbs = int(re.sub(r'lbs', '', w))
        return round(lbs * 0.453592)  # convert to kg
    elif "kg" in w:
        return int(re.sub(r'kg', '', w))
    else:
        return None

df['Weight_kg'] = df['Weight'].apply(convert_weight)


print to check the new results

In [ ]:
print(df[['Height', 'Height_cm', 'Weight', 'Weight_kg']].head(15))

2. remove the unnecessary newline characters from all columns that have them.

In [ ]:
# Clean all string columns by replacing \n and \r
df = df.map(lambda x: str(x).replace('\n', ' ').replace('\r', ' ') if isinstance(x, str) else x)
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
# Look for any remaining newline characters
print(df.map(lambda x: '\n' in str(x) or '\r' in str(x)).any())


3. Based on the joined colunn, check which players have been playing at a club for more than 10 years!

In [ ]:
from datetime import datetime
# Temporarily set display option
# print(df.columns.tolist())
print(df['Joined'].tolist())
def years_in_team(y):
    try:
        # join_year = pd.to_datetime(y).year
        # print()
        return datetime.now().year - pd.to_datetime(y).year
    except Exception as e:
        print(e)
        return "invalid start date"

def up_to_ten_years(years):
    try:
        return int(years) >= 10
    except:
        return False

df['Years_in_Team'] = df['Joined'].apply(years_in_team)
df['Ten_Years_or_More'] = df['Years_in_Team'].apply(up_to_ten_years)
print(df[['Joined', 'Years_in_Team', 'Ten_Years_or_More']].head(10))

4. Value', 'Wage' and "Release Clause' are string columns. Convert them to numbers. For eg, "M" in value column is Million, so multiply the row values by 1,000,000, etc.

In [ ]:
print(df['Wage'].tolist())
# df['Wage'] = df['Wage'].map(lambda x: str(x).replace('€', '').strip() if isinstance(x, str) else x)
# print(df['Wage'].tolist())
scale = {
    'K': 1000,
    'M': 1000000
}
def convert_value(v):
    for key, value in scale.items():
        if key in v:
            v = v.replace('€', '').replace(key,'').strip()
            # print(float(v) * value)
            # print(int(float(v) * value))
            return int(float(v) * value)
            
try:
    df['Wage'] = df['Wage'].map(convert_value)
    df['Value'] = df['Value'].map(convert_value)
    df['Release Clause'] = df['Release Clause'].map(convert_value)
except TypeError as e:
    print("conversion already done")
# print(df[['Wage', 'Value']].head(10))


5. Some columns have 'star' characters. Strip those columns of these stars and make the columns numerical

In [44]:
# print(df['SM'])
# df['SM'] = df['SM'].map(lambda x: x.replace('★', '') if isinstance(x, str) else x)
df['SM'] = df['SM'].map(lambda x: x.replace('★', '') if isinstance(x, str) else x)
df['W/F'] = df['W/F'].map(lambda x: x.replace('★', '') if isinstance(x, str) else x)
df['IR'] = df['IR'].map(lambda x: x.replace('★', '') if isinstance(x, str) else x)



# print(df['SM'])

# df = df.apply(pd.to_numeric, errors='coerce')

6. Which players are highly valuable but still underpaid (on low wages)? (hint: scatter plot between wage and value)

In [ ]:
x = df['Value'].apply(lambda v: float(str(v).replace('M', '').replace('€', '').replace('K', '')) if pd.notna(v) else 0)
# y = df['Wage'].apply(lambda w: int(str(w).replace('K', '').replace('€', '')) if pd.notna(w) else 0)
y = df['Wage'].apply(lambda w: int(str(w).replace('K', '').replace('€', '')) if pd.notna(w) else 0)

# plt.scatter(x, y)
# plt.xlabel('Value')
# plt.ylabel('Wage')
# plt.title('Scatter Plot of Player Value vs Wage')
# plt.show()

df['ratio'] = x / y
df_underpaid = df.nlargest(50, 'ratio')
print(df_underpaid[['Name', 'Value', 'Wage', 'ratio']])